<a href="https://colab.research.google.com/github/krishan30/AiLERT-FYP/blob/main/DCL_Framework_with_huggingface_hyper_parameter_tuning_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [ ]:
!pip install transformers
!pip install evaluate
!pip install accelerate -U
!pip install torchmetrics
!pip install optuna_dashboard
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 999.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,WeightedRandomSampler
from transformers import AutoModel,AutoTokenizer,AdamW,get_scheduler
from tqdm import tqdm
import evaluate
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
import random
from torchmetrics.classification import BinaryAccuracy
import optuna

## Setting up the GPU or CPU

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0"  if torch.cuda.is_available() else "cpu")

## Hyperparameter Values

In [ ]:
#@title #Hyperparameter Values
BATCH_SIZE = 144 # @param {type:"integer"}
EPOCHS = 2 # @param {type:"integer"}
DROPOUT =0.5# @param {type:"number"}
ALPHA = 0.4601079528693497 # @param {type:"number"}
GAMMA = 4.932651675660353 # @param {type:"number"}
TEMP_1 = 0.14630528484684996 # @param {type:"number"}
TEMP_2 =  0.07242998464641397# @param {type:"number"}
LAMBDA = 1
SEED = 1234 # @param {type:"integer"}


In [ ]:
{'epochs': 2, 'alpha': 0.4601079528693497, 'gamma': 4.932651675660353, 'temp_1': 0.14630528484684996, 'temp_2': 0.07242998464641397, 'batch_size': 144}

## Setting Random Seed for Reproducibility

In [ ]:
def setup_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    #torch.random.manual_seed(seed)
    torch.backends.cudnn.deterministic = True



In [ ]:
setup_seed(SEED)

In [ ]:
# @title Bert Model
bert_model_name = "bert-base-cased" # @param {type:"string"}

## Loading Train,Dev,Test Datasets

In [ ]:
dataset = load_dataset("krishan-CSE/HatEval_New")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Loading the Tokernizer for the Transformer Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer.batch_encode_plus(examples["text"], padding='max_length',max_length=45,add_special_tokens=True,truncation=True)

## Tokenize the Dataset

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 8982
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 998
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/8982 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8982
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 998
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3000
    })
})

## Remove Unwanted Coloumns

In [ ]:
tokenized_datasets=tokenized_datasets.remove_columns(['text'])

## Format the coloumns

In [ ]:
tokenized_datasets=tokenized_datasets.with_format("torch")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8982
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 998
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3000
    })
})

## Creating DataLoaders for Train,Dev,Test Datasets

In [ ]:
train_dataloader=DataLoader(tokenized_datasets["train"], batch_size=BATCH_SIZE ,shuffle=True)

In [ ]:
dev_dataloader=DataLoader(tokenized_datasets["validation"], batch_size=BATCH_SIZE , shuffle=True)

In [ ]:
test_dataloader=DataLoader(tokenized_datasets["test"], batch_size=BATCH_SIZE , shuffle=True)

In [ ]:
for batch in train_dataloader:
    break
batch.items()

dict_items([('labels', tensor([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 0, 1, 1, 0, 1])), ('input_ids', tensor([[  101, 12118,  4386,  ...,     0,     0,     0],
        [  101,   144, 17643,  ...,     0,     0,     0],
        [  101,  1422,  4906,  ...,     0,     0,     0],
        ...,
        [  101,  2009,  5718,  ...,     0,     0,     0],
        [  101,  2066,  2676,  ...,     0,     0,     0],
        [  101,  1706,  4392,  ...,     0,     0,     0]])), ('token_type_ids', tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 

## Define the Bert Architecture



In [ ]:
class DCLArchitecture(nn.Module):
    def __init__(self,dropout:float,bert_model_name:str='bert-base-cased'):
        super(DCLArchitecture, self).__init__()
        #model_class, pretrained_weights = (AutoModel, bert_model_name)
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.dim = 768
        self.dense = nn.Linear(self.dim, 1)
        self.dropout = nn.Dropout(dropout)
        #self.dropout_2 = nn.Dropout(0.7)

    def forward(self,batch_tokenized, if_train=False):
        input_ids = batch_tokenized['input_ids']
        attention_mask = batch_tokenized['attention_mask']
        bert_output = self.bert(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        #bert_cls_hidden_state = bert_output[0][:,0,:]
        bert_cls_hidden_state = bert_output[1]
        torch.cuda.empty_cache()
        # hidden_states = bert_output[2][-1]
        # bert_cls_hidden_state = torch.mean(hidden_states, 1)

        if if_train:
            bert_cls_hidden_state_aug = self.dropout(bert_cls_hidden_state)
            bert_cls_hidden_state = torch.cat((bert_cls_hidden_state, bert_cls_hidden_state_aug), dim=1).reshape(-1, self.dim)
        else:
            bert_cls_hidden_state = self.dropout(bert_cls_hidden_state)

        linear_output = self.dense(bert_cls_hidden_state)
        linear_output = linear_output.squeeze(1)

        return bert_cls_hidden_state, linear_output

## Define Focal Loss

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha:float=0.4, gamma:float=2, size_average:bool=True):
        super(FocalLoss, self).__init__()
        self.alpha = torch.tensor(alpha)
        self.gamma = gamma
        self.size_average = size_average

    def forward(self, pred, target):

        device = target.device
        self.alpha = self.alpha.to(device)

        pred = nn.Sigmoid()(pred)
        pred = pred.view(-1, 1)
        target = target.view(-1, 1)
        pred = torch.cat((1-pred, pred), dim=1)

        class_mask = torch.zeros(pred.shape[0], pred.shape[1]).to(device)
        class_mask.scatter_(1, target.view(-1, 1).long(), 1.)
        probs = (pred * class_mask).sum(dim=1).view(-1, 1)
        probs = probs.clamp(min=0.0001, max=1.0)

        log_p = probs.log()
        alpha = torch.ones(pred.shape[0], pred.shape[1]).to(device)
        alpha[:, 0] = alpha[:, 0] * (1 - self.alpha)
        alpha[:, 1] = alpha[:, 1] * self.alpha
        alpha = (alpha * class_mask).sum(dim=1).view(-1, 1)

        batch_loss = -alpha * (torch.pow((1 - probs), self.gamma)) * log_p

        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()

        return loss

## Define Unsupervised Contrastive loss

In [ ]:
def simcse_loss(batch_emb,temp_1:float):
    batch_size = batch_emb.size(0)
    y_true = torch.cat([torch.arange(1, batch_size, step=2, dtype=torch.long).unsqueeze(1),
                        torch.arange(0, batch_size, step=2, dtype=torch.long).unsqueeze(1)],
                       dim=1).reshape([batch_size,]).to(device)
    norm_emb = F.normalize(batch_emb, dim=1, p=2)
    sim_score = torch.matmul(norm_emb, norm_emb.transpose(0,1))
    sim_score = sim_score - (torch.eye(batch_size) * 1e12).to(device)
    sim_score = sim_score / temp_1
    loss_func = nn.CrossEntropyLoss()
    loss = loss_func(sim_score, y_true)

    return loss


## Define Supervised Contrastive loss

In [ ]:
def sup_simcse_loss(batch_emb, label,temp_2:float):
    n = batch_emb.size(0)

    similarity_matrix = F.cosine_similarity(batch_emb.unsqueeze(1), batch_emb.unsqueeze(0), dim=2)
    mask = torch.ones_like(similarity_matrix) * (label.expand(n, n).eq(label.expand(n, n).t()))

    mask_no_sim = torch.ones_like(mask) - mask
    mask_dui_jiao_0 = ((torch.ones(n,n) - torch.eye(n,n)) * 1e12).to(device)
    similarity_matrix = torch.exp(similarity_matrix/temp_2)
    similarity_matrix = similarity_matrix * mask_dui_jiao_0

    sim = mask*similarity_matrix
    no_sim = similarity_matrix - sim
    no_sim_sum = torch.sum(no_sim , dim=1)
    no_sim_sum_expend = no_sim_sum.repeat(n, 1).T

    sim_sum  = sim + no_sim_sum_expend
    loss = torch.div(sim , sim_sum)
    loss = mask_no_sim + loss + (torch.eye(n, n)/1e12).to(device)
    loss = -torch.log(loss)
    loss = torch.sum(torch.sum(loss, dim=1))/(2*n)
    return loss

## Fine Tuning the Model

## Format the Labels

In [ ]:
def copy_label(labels):
    labels = labels.unsqueeze(1)
    labels = torch.cat((labels, labels), dim=1).reshape(-1, 1).squeeze(1)
    return labels

### Train the Model

In [ ]:
def train_model(model, train_dataloader,optimizer,criteon,device,temp_1:float,temp_2:float,lamda:float):
    accuracy_metric = BinaryAccuracy()
    accuracy_metric.to(device)
    progress_bar = tqdm(range(len(train_dataloader)))
    model.train()
    total_train_loss = 0.0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = copy_label(batch["labels"])
        emb, pred = model(batch, if_train=True)
        loss = criteon(pred, labels.float())
        loss_sim = simcse_loss(emb,temp_1=TEMP_1)
        loss_supsim = sup_simcse_loss(emb, labels,temp_2=TEMP_2)
        total_train_loss += loss.item()
        pred = torch.sigmoid(pred)
        accuracy_metric(pred,labels)

        loss = loss + lamda*(loss_sim + loss_supsim)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar.update(1)
    average_epoch_train_loss = total_train_loss / len(train_dataloader)  # Compute average epoch loss
    train_accuracy = accuracy_metric.compute()
    return average_epoch_train_loss,train_accuracy


### Evaluate the Model

In [ ]:
def evaluate_model(model, dev_dataloader,criteon, device):
    accuracy_metric = BinaryAccuracy()
    accuracy_metric.to(device)
    model.eval()

    total_dev_loss = 0.0
    for batch in dev_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"]
        with torch.no_grad():
            emb, pred = model(batch , False)
            loss = criteon(pred, labels.float())
        total_dev_loss += loss.item()
        pred = torch.sigmoid(pred)
        accuracy_metric(pred,labels)

    average_epoch_dev_loss = total_dev_loss / len(dev_dataloader)  # Compute average epoch loss
    dev_accuracy = accuracy_metric.compute()
    return average_epoch_dev_loss,dev_accuracy

### Train & Evaluate Model

## Hyperparameter Tuning

In [ ]:
storage = optuna.storages.RDBStorage("sqlite:////content/drive/MyDrive/FYP/Hyper_Parameter_Tuning/optimization_study.db")

In [ ]:
def define_model(bert_model_name,trial):
    DROPOUT = 0.5
    model = DCLArchitecture(bert_model_name=bert_model_name,dropout=DROPOUT)
    model.to(device)
    return model

In [ ]:
def tokenized_batch_data(trial,tokenized_datasets):
    BATCH_SIZE = trial.suggest_int("batch_size",100,220)
    train_dataloader=DataLoader(tokenized_datasets["train"], batch_size=BATCH_SIZE , shuffle=True)
    dev_dataloader=DataLoader(tokenized_datasets["validation"], batch_size=BATCH_SIZE , shuffle=True)
    return train_dataloader,dev_dataloader


In [ ]:
checkpoint_path="/content/drive/MyDrive/FYP/Hyper_Parameter_Tuning/"

In [ ]:
def objective(trial):
    # Define the hyperparameters to optimize

    EPOCHS = trial.suggest_int("epochs", 1,2)
    ALPHA= trial.suggest_float("alpha", 0.1, 0.5)
    GAMMA = trial.suggest_float("gamma", 1.0, 5)
    TEMP_1 = trial.suggest_float("temp_1", 0.05, 0.15)
    TEMP_2 = trial.suggest_float("temp_2", 0.01,0.1)
    train_dataloader,dev_dataloader =tokenized_batch_data(trial,tokenized_datasets)
    model = define_model(bert_model_name,trial)
    optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-4)
    criteon = FocalLoss(ALPHA, GAMMA)


    for epoch in range(EPOCHS):
        average_epoch_train_loss,train_accuracy = train_model(model, train_dataloader,optimizer,criteon,device,TEMP_1,TEMP_2,LAMBDA)
        average_epoch_dev_loss,dev_accuracy = evaluate_model(model, dev_dataloader,criteon, device)

        trial.report(dev_accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    torch.save(model.state_dict(), checkpoint_path+'model_'+str(trial.number)+'.pt')
    torch.save(criteon.state_dict(),checkpoint_path+'criteon_'+str(trial.number)+'.pt')


    return dev_accuracy


In [ ]:
study = optuna.create_study(study_name='DCL-Experiments_1',storage=storage,load_if_exists=True,direction='maximize')
study.optimize(objective, n_trials=50)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-12-07 05:36:37,480] A new study created in RDB with name: DCL-Experiments_1


  0%|          | 0/79 [00:00<?, ?it/s]

[I 2023-12-07 05:37:52,159] Trial 0 finished with value: 0.6853707432746887 and parameters: {'epochs': 1, 'alpha': 0.39017422514975963, 'gamma': 4.258022814994655, 'temp_1': 0.09051337258511409, 'temp_2': 0.09840686596798195, 'batch_size': 114}. Best is trial 0 with value: 0.6853707432746887.


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

...

[I 2023-12-07 07:00:13,406] Trial 48 pruned. 


  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

[I 2023-12-07 07:02:28,187] Trial 49 pruned. 


Accuracy: 0.7805611491203308
Best hyperparameters: {'epochs': 2, 'alpha': 0.4601079528693497, 'gamma': 4.932651675660353, 'temp_1': 0.14630528484684996, 'temp_2': 0.07242998464641397, 'batch_size': 144}
